Grundlage:
* https://js.langchain.com/v0.1/docs/modules/chains/popular/sqlite/
* https://js.langchain.com/v0.1/docs/integrations/toolkits/sql/

* https://python.langchain.com/docs/tutorials/sql_qa/

# Setup

In [21]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase

In [22]:
db_path = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [23]:
print(db.dialect)
db.get_usable_table_names()

sqlite


['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [24]:
db.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

**Prompt Template** - Definieren, wie die Frage in eine SQL-Anweisung übersetzt wird.

In [25]:
sql_template = """
Du bist ein SQL-Experte, der ausschließlich lesende (SELECT-)Abfragen schreibt.
Deine Aufgabe ist es, aus einer natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist. Beachte bitte:
  - Verwende nur die Tabellen und Spalten, die im Schema aufgeführt sind.
  - Schreibe ausschließlich SELECT-Abfragen, keine INSERT-, UPDATE- oder DELETE-Anweisungen.
  - Wenn du die Antwort nicht sicher ableiten kannst, gib eine höfliche Meldung zurück, dass die Frage unklar ist.

Bitte verwende exakt das folgende Format:

Frage: <die ursprüngliche Frage>
SQL-Abfrage: <die generierte SQL-Abfrage>
SQL-Ergebnis: <das Ergebnis der SQL-Abfrage>
Antwort: <eine klare, kurze und präzise Antwort auf die ursprüngliche Frage>

Nutze das unten stehende Datenbankschema:
{dbschema}

Frage: {question}
SQL-Abfrage:
"""


In [26]:
from langchain import PromptTemplate #, SQLTemplate, SQLDatabase, SQLQuery, SQLResult, SQLAnswer

In [27]:
prompt_template = PromptTemplate(
        input_variables=["dbschema", "question"],
        template=sql_template
    )
    
# Example database schema and question
dbschema = "Album(AlbumID, Title, ArtistID); Artist(ArtistID, Name); Track(TrackID, Name, AlbumID)"
question = "Zeige alle Tracks aus dem Album Jazz"

**Integrate the LLM**

In [28]:
import requests
from typing import List, Optional, Mapping, Any
from langchain.llms.base import LLM
from pydantic import BaseModel

class OllamaLLM(LLM):
    model: str
    base_url: str = "http://localhost:11434"
    token: Optional[str] = None
    temperature: float = 0.7
    max_tokens: int = 256

    @property
    def _llm_type(self) -> str:
        return "ollama"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs: Any) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        payload = {
            "model": self.model,
            "prompt": prompt,
            "options": {
                "temperature": self.temperature,
                "num_predict": self.max_tokens,
            },
            "token": self.token or ""
        }
        url = f"{self.base_url}/api/generate"
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        # Adjust this key depending on your Ollama API response format
        return data.get("response", "").strip()

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model, "base_url": self.base_url}




In [8]:
!ollama list

NAME                     	ID          	SIZE  	MODIFIED     
qwen2.5-coder:7b         	2b0496514337	4.7 GB	31 hours ago	
deepseek-r1:8b           	28f8fd6cdc67	4.9 GB	3 days ago  	
llama3.2:1b-instruct-q4_0	53f2745c8077	770 MB	3 months ago	
llama3.2:1b              	baf6a787fdff	1.3 GB	3 months ago	
llama3.1:8b              	42182419e950	4.7 GB	4 months ago	
mistral:instruct         	f974a74358d6	4.1 GB	4 months ago	


#### Funktioniert, aber ist nicht gut optimiert

In [ ]:
import requests
import json
from langchain import PromptTemplate

def generate_sql(prompt, model="llama3.1:8B", token="your_api_token_here", temperature=0.7, max_tokens=256, base_url="http://localhost:11434"):
    # Prepare the payload with streaming disabled
    payload = {
        "model": model,
        "prompt": prompt,
        "options": {
            "temperature": temperature,
            "num_predict": max_tokens
        },
        "token": token,
        "stream": False  # Disable streaming to get a complete response
    }
    url = f"{base_url}/api/generate"
    response = requests.post(url, json=payload)
    response.raise_for_status()
    
    # Parse the complete JSON response
    data = response.json()
    # Assuming the API returns the generated SQL under the "response" key:
    return data.get("response", "").strip()

if __name__ == "__main__":
    # Define the SQL prompt template using LangChain's PromptTemplate
    sql_template = """
Du bist ein SQL-Experte, der ausschließlich SELECT-Abfragen generiert.
Deine Aufgabe ist es, aus der natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist.
Verwende nur die aufgeführten Tabellen und Spalten.
Falls die Frage unklar ist, gib eine höfliche Meldung zurück.

Datenbankschema:
{dbschema}

Frage: {question}
SQL-Abfrage:
    """
    
    prompt_template = PromptTemplate(
        input_variables=["dbschema", "question"],
        template=sql_template
    )
    
    # Example database schema and question
    dbschema = "Album(AlbumID, Title, ArtistID); Artist(ArtistID, Name); Track(TrackID, Name, AlbumID)"
    question = "Zeige alle Tracks aus dem Album Jazz"
    
    # Generate the final prompt using the template
    prompt = prompt_template.format(dbschema=dbschema, question=question)
    sql_query = generate_sql(prompt)
    print("Generated SQL Query:")
    print(sql_query)


Generated SQL Query:
Ich kann keine SQL-Abfragen generieren, die auf einem nicht existierenden Datenbankschema abzielen. Wenn Sie jedoch ein bestimmtes Szenario beschreiben, kann ich eine mögliche Lösung basierend auf dem gegebenen Schema erstellen.

Wenn ich jedoch das gegebene Szenario interpretiere und annehmen muss, dass es sich um die Tabelle "Album" handelt, von der wir alle Tracks aus dem Album "Jazz" herausfiltern möchten, könnte die SQL-Abfrage wie folgt aussehen:

```sql
SELECT t.Name AS TrackName
FROM Track t
JOIN Album a ON t.AlbumID = a.AlbumID
WHERE a.Title = 'Jazz';
```

In dieser Abfrage wird zunächst alle Tracks (Name) gefiltert, die sich mit der ID (`AlbumID`) mit denjenigen verbinden, die in der Tabelle `Album` unter dem Titel "Jazz" stehen.


### Versuch für bessere Schemaübergabe

In [12]:
import requests
import json
from langchain import PromptTemplate

def generate_sql(prompt, model="llama3.1:8B", token="your_api_token_here", temperature=0.0, max_tokens=256, base_url="http://localhost:11434"):
    # Prepare the payload with streaming disabled
    payload = {
        "model": model,
        "prompt": prompt,
        "options": {
            "temperature": temperature,
            "num_predict": max_tokens
        },
        "token": token,
        "stream": False  # Disable streaming to get a complete response
    }
    url = f"{base_url}/api/generate"
    response = requests.post(url, json=payload)
    response.raise_for_status()
    
    # Parse the complete JSON response
    data = response.json()
    # Assuming the API returns the generated SQL under the "response" key:
    return data.get("response", "").strip()


In [15]:
# Generate the final prompt using the template
prompt = prompt_template.format(dbschema=dbschema, question=question)
sql_query = generate_sql(prompt)
print("Generated SQL Query:")
print(sql_query)

Generated SQL Query:
Ich kann keine SQL-Abfragen generieren. Wenn du jedoch eine bestimmte Frage hast, kann ich versuchen, sie in eine korrekte SQL-Abfrage zu übersetzen. 

Um deine Frage zu beantworten: Um alle Tracks aus dem Album "Jazz" anzuzeigen, müsste ich wissen, welches AlbumID der Titel "Jazz" hat. Wenn wir jedoch davon ausgehen, dass wir das AlbumID kennen, könnten wir folgende Abfrage verwenden:

```sql
SELECT T1.Name 
FROM Track AS T1 
JOIN Album AS T2 ON T1.AlbumID = T2.AlbumID 
WHERE T2.Title = 'Jazz';
```

Dieser Code würde alle Tracks aus dem Album "Jazz" anzeigen. Wenn du jedoch wissen möchtest, wie man die Tracks aus einem bestimmten Album findet, ohne das AlbumID zu kennen, müsste ich mehr Informationen über deine Datenbank haben.

Bitte beachte, dass dies eine vereinfachte Antwort ist und je nachdem, was genau gefragt wird, könnte es weitere Abfragen geben.


In [30]:
import requests
import json
from langchain import PromptTemplate
from langchain_community.utilities import SQLDatabase

# Function to extract a simple schema string from the SQLDatabase object.
def extract_schema(db: SQLDatabase) -> str:
    # Get the list of usable table names
    tables = db.get_usable_table_names()
    schema_parts = []
    # For each table, try to extract the columns.
    # (Assumes that db provides a method get_table_columns; if not, you can implement one using PRAGMA queries.)
    for table in tables:
        try:
            # This method may vary depending on your SQLDatabase implementation.
            # For SQLite, one common approach is to use PRAGMA table_info(table)
            columns = db.get_table_columns(table)
        except Exception:
            # Fallback: if no column information is available, just list the table name.
            columns = []
        if columns:
            schema_parts.append(f"{table}({', '.join(columns)})")
        else:
            schema_parts.append(f"{table}")
    # Join all table schema strings with semicolons.
    return "; ".join(schema_parts)

In [ ]:
from sqlalchemy import text

def improved_extract_schema(db: SQLDatabase) -> str:
    """Extract a detailed schema string using PRAGMA table_info for each table.
    
    For each table in db.get_usable_table_names(), run the PRAGMA query and build a description
    including the column name, type, and NOT NULL constraint.
    """
    schema_parts = []
    # Use the underlying SQLAlchemy engine (stored in _engine) to get a connection.
    with db._engine.connect() as connection:
        for table in db.get_usable_table_names():
            # Wrap the SQL in text() to make it executable.
            result = connection.execute(text(f"PRAGMA table_info('{table}')"))
            columns = []
            for row in result:
                # row structure: (cid, name, type, notnull, dflt_value, pk)
                col_name = row[1]
                col_type = row[2]
                notnull = " NOT NULL" if row[3] else ""
                columns.append(f"{col_name} ({col_type}{notnull})")
            schema_parts.append(f"{table}: " + ", ".join(columns))
    return "\n".join(schema_parts)

# Example usage:
if __name__ == "__main__":
    db_path = "./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
    db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
    
    # Use our improved schema extraction:
    dbschema = improved_extract_schema(db)
    
    from langchain import PromptTemplate
    
    sql_template = """
Du bist ein SQL-Experte, der ausschließlich lesende (SELECT-)Abfragen schreibt.
Deine Aufgabe ist es, aus einer natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist. Beachte bitte:
  - Verwende nur die Tabellen und Spalten, die im Schema aufgeführt sind.
  - Schreibe ausschließlich SELECT-Abfragen, keine INSERT-, UPDATE- oder DELETE-Anweisungen.
  - Wenn du die Antwort nicht sicher ableiten kannst, gib eine höfliche Meldung zurück, dass die Frage unklar ist.

Bitte verwende exakt das folgende Format:

Frage: <die ursprüngliche Frage>
SQL-Abfrage: <die generierte SQL-Abfrage>
SQL-Ergebnis: <das Ergebnis der SQL-Abfrage>
Antwort: <eine klare, kurze und präzise Antwort auf die ursprüngliche Frage>

Nutze das unten stehende Datenbankschema:
{dbschema}

Frage: {question}
SQL-Abfrage:
    """
    
    prompt_template = PromptTemplate(
        input_variables=["dbschema", "question"],
        template=sql_template
    )
    

    prompt = prompt_template.format(dbschema=dbschema, question=question)
    print(prompt)
    question = "Zeige alle Tracks aus dem Album Jazz"


Du bist ein SQL-Experte, der ausschließlich lesende (SELECT-)Abfragen schreibt.
Deine Aufgabe ist es, aus einer natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist. Beachte bitte:
  - Verwende nur die Tabellen und Spalten, die im Schema aufgeführt sind.
  - Schreibe ausschließlich SELECT-Abfragen, keine INSERT-, UPDATE- oder DELETE-Anweisungen.
  - Wenn du die Antwort nicht sicher ableiten kannst, gib eine höfliche Meldung zurück, dass die Frage unklar ist.

Bitte verwende exakt das folgende Format:

Frage: <die ursprüngliche Frage>
SQL-Abfrage: <die generierte SQL-Abfrage>
SQL-Ergebnis: <das Ergebnis der SQL-Abfrage>
Antwort: <eine klare, kurze und präzise Antwort auf die ursprüngliche Frage>

Nutze das unten stehende Datenbankschema:
Album: AlbumId (INTEGER NOT NULL), Title (NVARCHAR(160) NOT NULL), ArtistId (INTEGER NOT NULL)
Artist: ArtistId (INTEGER NOT NULL), Name (NVARCHAR(120))
Customer: CustomerId (INTEGE

In [39]:
question = "Welche Kunden haben den höchsten Gesamtumsatz generiert?"

# Generate the final prompt using the template
prompt = prompt_template.format(dbschema=dbschema, question=question)
sql_query = generate_sql(prompt)
print("Generated SQL Query:")
print(sql_query)

Generated SQL Query:
Leider kann ich dir nicht helfen. Die Frage ist unklar. Das Datenbankschema enthält eine Tabelle namens "Invoice", aber es gibt keine direkte Beziehung zwischen diesem und der Tabelle "Customer". Es gibt jedoch die Möglichkeit, dass das Kundenid aus der Tabelle "Invoice" herangezogen werden kann, um den jeweiligen Kunden zu identifizieren.


In [44]:
from sqlalchemy import text

def extract_schema_with_relationships(db: SQLDatabase) -> str:
    """Extrahiert den Tabellenschema-String inklusive Fremdschlüssel-Relationen.
    
    Für jede benutzbare Tabelle wird PRAGMA table_info und PRAGMA foreign_key_list ausgeführt.
    Die Spalten-Informationen (Name, Typ, NOT NULL) werden gesammelt und falls vorhanden,
    werden die Fremdschlüsselbeziehungen (Format: <Spalte> -> <referenzierte Tabelle>(<referenzierte Spalte>))
    angehängt.
    """
    schema_parts = []
    # Verwenden des zugrundeliegenden SQLAlchemy-Engines (hier als _engine angenommen)
    with db._engine.connect() as connection:
        for table in db.get_usable_table_names():
            # Spalteninformationen abrufen
            result_columns = connection.execute(text(f"PRAGMA table_info('{table}')"))
            columns = []
            for row in result_columns:
                # row hat folgende Struktur: (cid, name, type, notnull, dflt_value, pk)
                col_name = row[1]
                col_type = row[2]
                notnull = " NOT NULL" if row[3] else ""
                columns.append(f"{col_name} ({col_type}{notnull})")
            # Fremdschlüsselinformationen abrufen
            result_fkeys = connection.execute(text(f"PRAGMA foreign_key_list('{table}')"))
            fkeys = []
            for row in result_fkeys:
                # row enthält z.B.: (id, seq, table, from, to, on_update, on_delete, match)
                # Wir verwenden row[3] als Spalte in der aktuellen Tabelle,
                # row[2] als referenzierte Tabelle und row[4] als referenzierte Spalte.
                fkeys.append(f"{row[3]} -> {row[2]}({row[4]})")
            # Kombinieren der Spalten- und Fremdschlüsselinfos
            if fkeys:
                schema_parts.append(f"{table}: " + ", ".join(columns) + " | FK: " + ", ".join(fkeys))
            else:
                schema_parts.append(f"{table}: " + ", ".join(columns))
    return "\n".join(schema_parts)

# Beispielhafter Einsatz:
if __name__ == "__main__":
    db_path = "./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
    db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
    
    # Verwenden Sie die erweiterte Schemaextraktion:
    dbschema = extract_schema_with_relationships(db)
    
    from langchain import PromptTemplate
    sql_template = """
Du bist ein SQL-Experte, der ausschließlich lesende (SELECT-)Abfragen schreibt.
Deine Aufgabe ist es, aus einer natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist. Beachte bitte:
  - Verwende nur die Tabellen, Spalten und Relationen, die im Schema aufgeführt sind.
  - Schreibe ausschließlich SELECT-Abfragen, keine INSERT-, UPDATE- oder DELETE-Anweisungen.
  - Falls du die Antwort nicht sicher ableiten kannst, gib eine höfliche Meldung zurück, dass die Frage unklar ist.

Bitte verwende exakt das folgende Format:

Frage: <die ursprüngliche Frage>
SQL-Abfrage: <die generierte SQL-Abfrage>
SQL-Ergebnis: <das Ergebnis der SQL-Abfrage>
Antwort: <eine klare, kurze und präzise Antwort auf die ursprüngliche Frage>

Nutze das unten stehende Datenbankschema:
{dbschema}

Frage: {question}
SQL-Abfrage:
    """
    
    prompt_template = PromptTemplate(
        input_variables=["dbschema", "question"],
        template=sql_template
    )
    
    #question = "Zeige alle Tracks aus dem Album Jazz"
    prompt = prompt_template.format(dbschema=dbschema, question=question)
    
    print("Finaler Prompt für das Modell:")
    print(prompt)


Finaler Prompt für das Modell:

Du bist ein SQL-Experte, der ausschließlich lesende (SELECT-)Abfragen schreibt.
Deine Aufgabe ist es, aus einer natürlichen Spracheingabe eine korrekte SQL-Abfrage zu erstellen,
die exakt auf das folgende Datenbankschema abgestimmt ist. Beachte bitte:
  - Verwende nur die Tabellen, Spalten und Relationen, die im Schema aufgeführt sind.
  - Schreibe ausschließlich SELECT-Abfragen, keine INSERT-, UPDATE- oder DELETE-Anweisungen.
  - Falls du die Antwort nicht sicher ableiten kannst, gib eine höfliche Meldung zurück, dass die Frage unklar ist.

Bitte verwende exakt das folgende Format:

Frage: <die ursprüngliche Frage>
SQL-Abfrage: <die generierte SQL-Abfrage>
SQL-Ergebnis: <das Ergebnis der SQL-Abfrage>
Antwort: <eine klare, kurze und präzise Antwort auf die ursprüngliche Frage>

Nutze das unten stehende Datenbankschema:
Album: AlbumId (INTEGER NOT NULL), Title (NVARCHAR(160) NOT NULL), ArtistId (INTEGER NOT NULL) | FK: ArtistId -> Artist(ArtistId)
Artist:

In [45]:
question = "Welche Kunden haben den höchsten Gesamtumsatz generiert?"

# Generate the final prompt using the template
prompt = prompt_template.format(dbschema=dbschema, question=question)
sql_query = generate_sql(prompt)
print("Generated SQL Query:")
print(sql_query)

Generated SQL Query:
Ich kann diese Anfrage nicht bearbeiten, da sie unklar ist. Es fehlt eine genaue Beschreibung der geforderten Daten und Beziehungen zwischen den Tabellen. Bitte präziseiere deine Frage, damit ich eine korrekte SQL-Abfrage erstellen kann.
